# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cockburn town,21.4612,-71.1419,27.19,79,10,7.38,TC,1701834678
1,1,badger,64.8000,-147.5333,-17.58,83,100,1.54,US,1701834412
2,2,sirjan,29.4520,55.6814,8.99,20,0,0.76,IR,1701834679
3,3,iqaluit,63.7506,-68.5145,-14.15,84,20,1.03,CA,1701834679
4,4,blackmans bay,-43.0167,147.3167,22.07,37,8,1.34,AU,1701834679


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_cities = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_cities

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df.loc[((city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <27)) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness']  == 0)]

# Drop any rows with null values
vacation_df = vacation_df.dropna()

# Display sample data
vacation_df.sample(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
526,526,los angeles,34.0522,-118.2437,22.70,46,0,3.09,US,1701834095
362,362,kamakwie,9.5000,-12.2333,22.10,95,0,0.59,SL,1701834740
218,218,san pedro de macoris,18.4616,-69.2972,24.07,90,0,3.32,DO,1701834715
216,216,carlsbad,33.1581,-117.3506,21.04,57,0,0.89,US,1701834576
269,269,saint-philippe,-21.3585,55.7679,25.06,71,0,2.68,RE,1701834722
37,37,kayes,14.0000,-11.0000,23.10,14,0,2.60,ML,1701834686
330,330,menaka,15.9182,2.4022,22.39,17,0,4.18,ML,1701834733
505,505,puerto lempira,15.2667,-83.7667,25.43,89,0,2.99,HN,1701834763
62,62,nova vicosa,-17.8919,-39.3719,24.26,89,0,4.05,BR,1701834690
327,327,gonaives,19.4500,-72.6833,24.01,77,0,1.96,HT,1701834732


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
362,kamakwie,SL,9.5000,-12.2333,95,
526,los angeles,US,34.0522,-118.2437,46,
327,gonaives,HT,19.4500,-72.6833,77,
37,kayes,ML,14.0000,-11.0000,14,
269,saint-philippe,RE,-21.3585,55.7679,71,
505,puerto lempira,HN,15.2667,-83.7667,89,
133,sur,OM,22.5667,59.5289,39,
216,carlsbad,US,33.1581,-117.3506,57,
182,saint croix,VI,17.7397,-64.7388,82,
62,nova vicosa,BR,-17.8919,-39.3719,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": 'accommodation.hotel',
    "limit":20,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kayes - nearest hotel: No hotel found
nova vicosa - nearest hotel: Pousada Pontal da Barra
sur - nearest hotel: Sur Hotel
saint croix - nearest hotel: The Buccaneer
carlsbad - nearest hotel: Carlsbad Inn Beach Resort
san pedro de macoris - nearest hotel: No hotel found
saint-philippe - nearest hotel: Le Baril
gonaives - nearest hotel: amiral kilik
menaka - nearest hotel: No hotel found
kamakwie - nearest hotel: No hotel found
puerto lempira - nearest hotel: Hotel Yu Baiwan
los angeles - nearest hotel: DoubleTree by Hilton Hotel Los Angeles Downtown


,City,Country,Lat,Lng,Humidity,Hotel Name
37,kayes,ML,14.0000,-11.0000,14,No hotel found
62,nova vicosa,BR,-17.8919,-39.3719,89,Pousada Pontal da Barra
133,sur,OM,22.5667,59.5289,39,Sur Hotel
182,saint croix,VI,17.7397,-64.7388,82,The Buccaneer
216,carlsbad,US,33.1581,-117.3506,57,Carlsbad Inn Beach Resort
218,san pedro de macoris,DO,18.4616,-69.2972,90,No hotel found
269,saint-philippe,RE,-21.3585,55.7679,71,Le Baril
327,gonaives,HT,19.4500,-72.6833,77,amiral kilik
330,menaka,ML,15.9182,2.4022,17,No hotel found
362,kamakwie,SL,9.5000,-12.2333,95,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

hotel_df.set_index('City')

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols= 'all'
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)